In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Simple Impact-T Run

In [2]:
from impact import Impact

In [3]:
ipath = '../templates/lcls_injector/'
ifile = '../templates/lcls_injector/ImpactT.in'

In [4]:
I = Impact(ifile, verbose=True)

/Users/chrisonian/Code/GitHub/lume-impact/templates/lcls_injector


In [5]:
# Change some things
I.input['header']['Np'] = 30004
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13

In [6]:
lat = I.input['lattice']
stopeles = [e for e in lat if e['type']=='stop']
stopeles[0]['s'] = 0.001

In [7]:
I.timeout = 1000
I.run()

Loaded fort 30 : Fourth root of the fourth moments of the beam distribution
Loaded fort 25 : RMS Y information
Loaded fort 24 : RMS X information
Loaded fort 26 : RMS Z information
Loaded fort 28 : Load balance and loss diagnostics
Loaded fort 29 : Cube root of third moments of the beam distribution
Loaded fort 18 : Time and energy
Loaded fort 27 : Max amplitude information
Loaded fort 70 : Slice information of the final distribution
Loaded fort 60 : Slice information of the initial distribution
Loaded fort 40 : initial particle distribution at t = 0
Loaded fort 50 : final particle distribution projected to the centroid location of the bunch


In [ ]:
I.particles

In [ ]:
I.tempdir

In [ ]:
I.output['stats'].keys()

In [ ]:
I.output['slice_info']

# Plot

In [21]:
from bokeh.plotting import output_notebook
from bokeh.plotting import show
output_notebook(hide_banner=True)

from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Range1d
from impact.parsers import load_many_fort
from impact.parsers import UNITS


ds = ColumnDataSource(I.output['stats'])

In [22]:
def plot1(y_axis, x_axis='z', source=ds):
    yunit = UNITS[y_axis]
    ylabel = y_axis+' ('+yunit+')'
    
    xunit = UNITS[x_axis]
    xlabel = x_axis+' ('+xunit+')'
    p = figure(plot_width=600, plot_height=200, x_axis_label=xlabel, y_axis_label=ylabel)
    p.line( x_axis, y_axis, source=source, color='red')
    return p
show(plot1('x_rms'))